In [3]:
import os
import sys

file_dir = os.getcwd()
chatbot_dir = os.path.join(file_dir, '..')
bloomberg_dir = os.path.join(chatbot_dir, '..')
dirs = [chatbot_dir, bloomberg_dir, file_dir]

sys.path.extend(d for d in dirs if d not in sys.path)

import polars as pl 
import pandas as pd 
import json 

data_dir = os.path.join(bloomberg_dir,'data')

from Flex import Flex
from helpers.Database import Database
import warnings 

warnings.filterwarnings('ignore')

sym = 'KBANK'
flex = Flex(sym)

broker_list = pl.read_csv(os.path.join(data_dir, 'broker.csv')).to_pandas()
type_dict = broker_list[['broker', 'type']].set_index('broker').to_dict()['type']

In [4]:
db = Database()
year_df = db.query(f"SELECT * FROM bloomberg WHERE symbol = '{sym}' AND YEAR(date) = YEAR(CURDATE())")
year_df['date'] = pd.to_datetime(year_df['date'])


year_df['type'] = year_df['broker'].map(type_dict)

# ---------------------------------- Average --------------------------------- #
avg_sep = year_df.pivot_table(index='type', values ='target', aggfunc='mean').rename(columns = {'target': "Average Target Price"}).round(2)
avg_sep_local = avg_sep.loc[['local']].T.reset_index().T.reset_index(drop=True)
avg_sep_foreign = avg_sep.loc[['foreign']].T.reset_index().T.reset_index(drop=True)
# ----------------------------------- Mode ----------------------------------- #
mode_sep = year_df.pivot_table(index=['type','comment'] , aggfunc='count')['analyst'].reset_index()\
            .groupby('type').apply(lambda x: x.sort_values('analyst', ascending=False).head(1))
            
mode_sep['Most Common Comment'] = mode_sep['comment'] + ' (' + mode_sep['analyst'].astype(str) + ')'
mode_sep_foreign = mode_sep.loc[['foreign'], ['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T
mode_sep_local = mode_sep.loc[['local'], ['Most Common Comment']].T.reset_index().T.reset_index(drop=True).T

In [72]:
# ------------------------------- IM Component ------------------------------- #
im = pl.read_csv(os.path.join(data_dir, 'im.csv')).to_pandas()
im_df = im[im['symbol'] == sym]

im_df.drop(columns=['symbol'], inplace=True)
im_df.columns = ['ขั้นต่ำ', 'Leverage', 'ใช้เงิน']

im_df['ขั้นต่ำ'] = im_df['ขั้นต่ำ'].apply(lambda x: f'{x:,.0f}')
im_df['ใช้เงิน'] = im_df['ใช้เงิน'].apply(lambda x: f'{x:,.0f}')
im_df['Leverage'] = im_df['Leverage'].apply(lambda x: f'{x}')
im_df = im_df.T.reset_index()
im_df = im_df.T.reset_index(drop=True).T

In [76]:
im_df[2] = ['สัญญา', 'เท่า', 'บาท']

In [77]:
im_df

,0,1,2
0,ขั้นต่ำ,20,สัญญา
1,Leverage,19.07,เท่า
2,ใช้เงิน,"137,900",บาท
